<a href="https://colab.research.google.com/github/Elbereth-Elentari/Library_search/blob/master/Library_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import os
import random
import string
import re
import pandas as pd

!pip3 install spacy==2.3.0
import spacy
!pip3 install spacy_langdetect
from spacy_langdetect import LanguageDetector
import spacy.cli
spacy.cli.download('en_core_web_lg')
spacy.cli.download('pl_core_news_lg')
import en_core_web_lg
nlp = en_core_web_lg.load()
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)
import pl_core_news_lg
nlp_pl = pl_core_news_lg.load()

!pip3 install selenium
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium .webdriver import ChromeOptions
!apt-get install chromium-chromedriver

from ipykernel import kernelapp as app

!pip3 install gensim
import gensim
from gensim import corpora, models

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')
✔ Download and installation successful
You can now load the model via spacy.load('pl_core_news_lg')
Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (83.0.4103.61-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [2]:
def get_conditions():
    term = input('What would you like to read about?\n')
    term = term.replace(' ', '+')
    min_year = int(input('What is the oldest book you are interested in?\n'))
    min_length = 100
    max_length = 700
    return term, min_year, min_length, max_length

In [18]:
def next_or_break(driver):
    try:
        next_button = driver.find_element_by_link_text('Następne>')
        next_button.click()
        return True
    except: return 'no next'

In [16]:
class Book:
    interesting_books = set()
    all_authors = set()
    authors_to_scrape = set()

    def __init__(self, source):
        self.title = ''
        self.author = ''
        self.publisher = ''
        self.year = 0
        self.pages = 0
        self.WD_signature = ''
        self.storage = ''
        self.source = source

    def get_book_attributes(self, record):
        title = record.find_element_by_class_name('title').text
        self.title = re.sub(r' / .+', '', title)

        try: self.author = record.find_element_by_class_name('author').text
        except: pass

        if 'BUW Magazyn' in record.text: self.storage = 'magazyn'

        infos = record.find_elements_by_tag_name('tr')
        for info in infos:
            if 'Klasyfikacja WD' in info.text:
                self.WD_signature = info.find_element_by_tag_name('a').text
            elif 'Adres wyd.' in info.text:
                publisher_candidates = info.find_elements_by_tag_name('span')
                for publisher in publisher_candidates:
                    if publisher.get_attribute('class') != 'highlight':
                        publisher_with_colon = re.search(r'.+ : ?(.+),?.*(\d{4})', publisher.text)
                        if publisher_with_colon:
                            publisher = publisher_with_colon
                        else:
                            publisher = re.search(r'.+? (.+),?.*(\d{4})', publisher.text)
                        if publisher:
                            self.publisher = publisher.group(1)
                            self.year = int(publisher.group(2))
            elif 'Opis fiz.' in info.text:
                pages = info.find_element_by_tag_name('span').text
                pages = re.sub(r'\[.+?\]', '', pages)
                if re.search(r'\d+', pages):
                    self.pages = int(re.search(r'\d+', pages).group(0))

    def check_quality(self):
        if self.year >= min_year and self.pages >= min_length and self.pages <= max_length:
            Book.interesting_books.add(self)

    def add_to_authors(self, record):
        if self in Book.interesting_books and len(self.author) > 0:
            if self.author in Book.all_authors:
                author = record.find_element_by_class_name('author')
                Book.authors_to_scrape.add(author.get_attribute('href'))
            else: Book.all_authors.add(self.author)

In [17]:
def get_books(driver, source, expand_set):
    records = driver.find_element_by_class_name('records').find_elements_by_tag_name('li')
    for record in records:
        too_old = 'not too old'

        if ('BUW Wolny Dostęp' in record.text or 'BUW Magazyn' in record.text) and 'Adres wyd.' in record.text and 'Opis fiz.' in record.text:
            book = Book(source)
            book.get_book_attributes(record)
            book.check_quality()
            if expand_set: book.add_to_authors(record)
            if book.year < min_year and book.year > 0:
                too_old = 'too old'
                break
    return too_old

In [19]:
def create_link_set(driver):
    links = set()
    while True:
        tags_from_page = driver.find_element_by_tag_name('tbody').find_elements_by_tag_name('a')
        for tag in tags_from_page:
            try: links.add(tag.get_attribute('href'))
            except: pass
        if next_or_break(driver) == 'no next': break
    return links

In [20]:
def get_books_from_links(link_set, source, expand_set=True):
    counter = 1
    for link in link_set:
        print('Progress:', counter/len(link_set))
        driver.get(link)
        select = Select(driver.find_element_by_id('search_sort'))
        select.select_by_value('5')
        too_old = get_books(driver, source, expand_set)
        while too_old == 'not too old':
            if next_or_break(driver) == 'no next': break
            too_old = get_books(driver, source, expand_set)
        counter += 1
    return True

In [21]:
def deduplicate_books(df):
    df.drop_duplicates(inplace=True)
    df.sort_values(by='year', ascending=False, inplace=True)
    df.drop_duplicates(subset='title', inplace=True)
    return df

In [ ]:
term, min_year, min_length, max_length = get_conditions()

options = ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome('chromedriver', options=options)
driver.get(f'https://chamo.buw.uw.edu.pl/heading/search?match_1=MUST&field_1=heading&term_1={term}&facet_heading_type=subject&sort=heading')

print('Creating initial link set')
links = create_link_set(driver)
print(f'Scraping the {len(links)} initial links')
get_books_from_links(links, 'tags')
len_from_tags = len(Book.interesting_books)
print(f'Scraping {len(Book.authors_to_scrape)} interesting authors')
get_books_from_links(Book.authors_to_scrape, 'authors', False)
driver.close()
print(f'Scraped {len_from_tags} books from tags and {len(Book.interesting_books)-len_from_tags} books from authors.')

interesting_books = [{'title':book.title, 'author':book.author, 'publisher':book.publisher, 'year':book.year, 'pages':book.pages, 'WD_signature':book.WD_signature, 'storage':book.storage, 'source':book.source} for book in Book.interesting_books]
reading_list = pd.DataFrame(columns=['source', 'title','author', 'WD_signature', 'storage', 'publisher', 'year', 'pages'], data=interesting_books)
reading_list = deduplicate_books(reading_list)
output_file = f'/content/drive/My Drive/Library_search/data/results/{term}_reading_list.tsv'
reading_list.to_csv(output_file, index=False, sep='\t')
print(f'Scraped {len(reading_list)} books, without duplicates.')

In [75]:
def merge_tsvs():
    cat = pd.DataFrame()
    data_path = '/content/drive/My Drive/Library_search/data/scraped/'
    for file in os.listdir(data_path):
        if file.endswith('.tsv'):
            print('Processing', file)
            c = pd.read_csv(data_path + file, sep='\t')
            cat = cat.append(c, ignore_index=True)
    return cat

In [54]:
def detect_language(df):
    print('Detecting language')
    language = []
    for index, doc in enumerate(nlp.pipe(df['title'].values, batch_size=1000)):
        if index % 10000 == 0: print('Progress:', index/len(df['title']))
        if doc.is_parsed:
            if doc._.language['language'] == 'en': language.append('en')
            else: language.append('pl')
        else: language.append('')

    df['language'] = language
    for language in df.language.unique():
        df_lang = df[df['language'] == language]
        sample = random.sample(list(df_lang.index), min(5, len(df_lang)))
        print(f'{language} sample')
        for row in df[df.index.isin(sample)].itertuples(): print(row.title)
    return df

In [78]:
def preprocess(input_df):
    input_df = deduplicate_books(input_df)
    input_df = detect_language(input_df)
    preprocessed_df = pd.DataFrame()

    for lang in ['pl', 'en']:
        print(f'Preprocessing {lang}')
        if lang == 'en': model = nlp
        elif lang == 'pl': model = nlp_pl

        df = input_df[input_df['language'] == lang]
        stopwords = model.Defaults.stop_words
        preprocessed = []
        for index, doc in enumerate(model.pipe(df['title'].values, batch_size=200)):
            if index % 10000 == 0: print('Progress:', index/len(df))
            if doc.is_parsed:
                tokens = [token.lemma_.lower() for token in doc if (token.text.lower() not in stopwords and token.text not in string.punctuation)]
                if len(tokens) > 0: preprocessed.append(tokens)
                else: preprocessed.append('preprocessing_fail')
            else:
                print('Preprocessing failed')
                preprocessed.append('preprocessing_fail')

        df['tokens'] = preprocessed
        df = df[df['tokens'] != 'preprocessing_fail']
        preprocessed_df = preprocessed_df.append(df)
    return preprocessed_df

In [79]:
cat = merge_tsvs()
preprocessed_cat = preprocess(cat)

Processing BUW_catalogue_2525.tsv
Processing BUW_catalogue_5488.tsv
Processing BUW_catalogue_8214.tsv
Processing BUW_catalogue_11033.tsv
Processing BUW_catalogue_13816.tsv
Processing BUW_catalogue_16633.tsv
Processing BUW_catalogue_19430.tsv
Processing BUW_catalogue_22256.tsv
Processing BUW_catalogue_25078.tsv
Processing BUW_catalogue_27914.tsv
Processing BUW_catalogue_30769.tsv
Processing BUW_catalogue_33943.tsv
Processing BUW_catalogue_37144.tsv
Processing BUW_catalogue_40116.tsv
Processing BUW_catalogue_43040.tsv
Processing BUW_catalogue_46000.tsv
Processing BUW_catalogue_48881.tsv
Processing BUW_catalogue_51738.tsv
Processing BUW_catalogue_54552.tsv
Processing BUW_catalogue_57416.tsv
Processing BUW_catalogue_60377.tsv
Processing BUW_catalogue_63198.tsv
Processing BUW_catalogue_66104.tsv
Processing BUW_catalogue_68924.tsv
Processing BUW_catalogue_71929.tsv
Processing BUW_catalogue_74798.tsv
Processing BUW_catalogue_78001.tsv
Processing BUW_catalogue_81161.tsv
Processing BUW_catalogu

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Preprocessing en
Progress: 0.0
Progress: 0.8439530762089628


In [80]:
tag = preprocess(reading_list[reading_list['source'] == 'tags'])
tag['tokens'] = tag['tokens'].apply(lambda x: x+[term.replace('+', ' ')])
for index, row in tag.iterrows():
    preprocessed_cat.drop(preprocessed_cat[preprocessed_cat['title'] == row['title']].index, inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from 

Detecting language
Progress: 0.0
Progress: 0.970873786407767
pl sample
Strażnik wrót : próby z hermeneutyki teologicznej
Twój Kościół to szczyt wszystkiego! : fakty, argumenty, opinie
Status prawny ordynariatów personalnych dla anglikanów powracających do pełnej wspólnoty z Kościołem katolickim
Gender : kobieta w kulturze i społeczeństwie
Antyrzymski resentyment : papiestwo a Kościół
en sample
Nuns : a history of convent life 1450-1700
The Blackwell Companion to Catholicism
Latin books and the Eastern Orthodox clerical elite in Kiev, 1632-1780
Religious fundamentalisms and the human rights of women
Leimōn : studies presented to Lennart Rydén on His sixty-fifth birthday
Preprocessing pl
Progress: 0.0
Preprocessing en
Progress: 0.0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [83]:
cat_sim = pd.DataFrame()
for lang in ['pl', 'en']:
    print(f'Calculating semantic similarity for {lang}')
    df = preprocessed_cat[preprocessed_cat['language'] == lang]
    semantics = ' '.join(tag[tag['language'] == lang].tokens.str.join(' '))
    if lang == 'en': model = nlp
    elif lang == 'pl': model = nlp_pl
    semantics = model(semantics)

    similarity = []
    for index, doc in enumerate(model.pipe(df['title'].values, batch_size=200)):
        if index % 1000 == 0: print('Progress:', index/len(df))
        if doc.is_parsed: similarity.append(semantics.similarity(doc))
        else:
            print('Similarity failed')
            preprocessed.append('similarity_fail')
    df['similarity'] = similarity
    cat_sim = cat_sim.append(df)

Calculating semantic similarity for pl
Progress: 0.0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  del sys.path[0]


Progress: 0.008839075786235792
Progress: 0.017678151572471584
Progress: 0.026517227358707374
Progress: 0.03535630314494317
Progress: 0.044195378931178954
Progress: 0.05303445471741475
Progress: 0.06187353050365054
Progress: 0.07071260628988633
Progress: 0.07955168207612212
Progress: 0.08839075786235791
Progress: 0.09722983364859371
Progress: 0.1060689094348295
Progress: 0.11490798522106528
Progress: 0.12374706100730108
Progress: 0.13258613679353687
Progress: 0.14142521257977267
Progress: 0.15026428836600844
Progress: 0.15910336415224424
Progress: 0.16794243993848004
Progress: 0.17678151572471582
Progress: 0.18562059151095162
Progress: 0.19445966729718742
Progress: 0.2032987430834232
Progress: 0.212137818869659
Progress: 0.2209768946558948
Progress: 0.22981597044213056
Progress: 0.23865504622836636
Progress: 0.24749412201460216
Progress: 0.25633319780083796
Progress: 0.26517227358707374
Progress: 0.2740113493733095
Progress: 0.28285042515954534
Progress: 0.2916895009457811
Progress: 0.3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Progress: 0.0
Progress: 0.08443806467955754
Progress: 0.16887612935911508
Progress: 0.2533141940386726
Progress: 0.33775225871823017
Progress: 0.4221903233977877
Progress: 0.5066283880773452
Progress: 0.5910664527569028
Progress: 0.6755045174364603
Progress: 0.759942582116018
Progress: 0.8443806467955755
Progress: 0.928818711475133


In [84]:
cat_sim.drop(cat_sim[cat_sim['similarity'] < 0.7].index, inplace=True)
cat_sim.sort_values(by='similarity', inplace=True, ascending=False)
cat_sim['source'] = 'similarity'
for row in cat_sim[:20].itertuples(): print(row.title, row.similarity)
bibliography = reading_list.append(cat_sim.drop(columns={'language', 'tokens'}), ignore_index=True)

Ikonografia Chrystusa w prawosławiu 0.8703955568182945
Ikonografia świętych w prawosławiu 0.8581606047990691
Eucharystia w życiu Kościoła 0.8578219538157554
Katechumenat i inicjacja chrześcijańska w Kościele starożytnym 0.853117222748516
Maryja wychowawczyni w Kościele domowym 0.8524908189795436
Duchowieństwo katolickie w życiu Radziwiłłów nieświeskich w XVIII wieku 0.852106145513599
Historia miłości w zachodnim chrześcijaństwie 0.8515490268669106
Elementy duchowości w liturgicznych obrzędach sakramentu małżeństwa w Kościele Zachodnim 0.8460053877719586
Duchowość i historia benedyktynek w symbolice dekoracji malarskiej kościoła w Staniątkach 0.8453901050578992
Modlitwa w doświadczeniu Kościoła 0.8453757406314939
Religia w świecie człowieka 0.8414433190307946
Eucharystia w świetle psychologii pastoralnej 0.8391421928177744
Kultura niezależna w Kościele 0.8381544281088225
Autentyczność w życiu publicznym 0.8374401598419765
Kościół w świecie późnego antyku 0.836336567295731
Człowiek w ren

In [121]:
dictionary = corpora.Dictionary(tag.tokens)
corpus = [dictionary.doc2bow(doc) for doc in tag.tokens]
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=2, id2word=dictionary, passes=1000)
for topic, distribution in ldamodel.print_topics(num_words=5):
    print(f'Topic {topic}')
    print(distribution)

Topic 0
0.047*"kobieta w kościele" + 0.009*"współczesny" + 0.009*"kościół" + 0.007*"wiara" + 0.007*"kobiecy"
Topic 1
0.118*"kobieta w kościele" + 0.039*"kobieta" + 0.031*"kościół" + 0.010*"katolicyzm" + 0.007*"religia"


In [93]:
for source in ['tags', 'authors', 'similarity']:
    print(f'********** Bibliography for {source} **********')
    subcat = bibliography[bibliography['source'] == source]
    for row in subcat[:min(1000, len(subcat))].itertuples():
        print(row.year, row.WD_signature, row.storage, row.title)
    print('================================================')

********** Bibliography for tags **********
2019 BX1754 .W45165 2019  Światło Chrystusa : wprowadzenie do katolicyzmu
2019  magazyn Otulona : dziesięć Bożych darów dla kobiet
2019 BR65.A84 P6 2019 magazyn Kazania na Boże Narodzenie i Objawienie Pańskie
2019 BR118 .K55 2019  Strażnik wrót : próby z hermeneutyki teologicznej
2019 BL625.7 .S3 2019  Sacrum w doświadczeniu kobiet : kulturowe perspektywy polska i światowa : zbiorowa praca monograficzna
2019 BR115.W6 D96 2019  Dziedziniec Dialogu 2013-2015-2017
2018  magazyn Siła powołania : o życiu konsekrowanym i poprawie relacji w Kościele
2018  magazyn Sakramenty inicjacji chrześcijańskiej w parafii
2018 BX2015 .B49 2018  Tradycja anglikańska w mszale dla Ordynariatów Personalnych byłych anglikanów
2018  magazyn Twój Kościół to szczyt wszystkiego! : fakty, argumenty, opinie
2018 BX1378.7 .F685165 2018  Otwieranie drzwi : rozmowy o Kościele i świecie
2018 BX342.9.E27 K35 2018  Oikonomia kościelna w teorii i praktyce prawosławia
2018 BV676 

In [94]:
bibliography.to_csv(f'/content/drive/My Drive/Library_search/data/results/{term}_Bibliography', index=False, sep='\t')